In [2]:
# Fine tuning gpt2_medium model and use own data like company profile
#
# See also medium.com blog
# "GPT-2 Fine-Tuning Guide: Building a Chatbot for Your Company Profile"
# https://medium.com/@datatec.studio
#

# Mount google driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Change to google driver folder which contains datasets
# This folder will also be used to save model
%cd /content/drive/MyDrive/GPT2_Lab_DTS

/content/drive/MyDrive/GPT2_Lab_DTS


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 122.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:0

In [7]:
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm

In [8]:
# Define environment variable, path of data, model name and device
os.environ["HF_HOME"] = "/content/huggingface"  # Replace with your desired directory
print("Please replace it with your hf access token:")
os.environ["HF_HOME_TOKEN"] = "Please_replace_it_with_your_hf_access_token"

result_dir = '/content/drive/MyDrive/GPT2_Lab_DTS/results'
data_file_path = '/content/drive/MyDrive/GPT2_Lab_DTS/data/my_company_info.json'

model_name = "gpt2-medium"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

Please replace it with your hf access token:
cuda


In [9]:
# Write a python file to google driver
# Sample of json datasets
# You can also directly upload this code to your google driver
# The code write here in this way is for better understanding of whole project
%%writefile chat_data.py

from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path: str, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> " + i + " <bot>: " + self.X[idx + 1] + " <endofstring>"
            except:
                break

        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        total_samples = len(self.X)  # Calculate the total number of samples
        print("total_samples", total_samples)
        # define samples amount
        self.X = self.X[:500]
        print("Here is the self.X[0] i wanna check:")
        print(self.X[0])

        self.X_encoded = tokenizer(self.X, return_tensors="pt", max_length=30, padding="max_length", truncation=True)
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx]


Writing chat_data.py


In [10]:
# Download model, save model and tokernize to harddisk
## prepare tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"pad_token": "<pad>",
                            "bos_token": "<startofstring>",
                            "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

## prepare model
### Specify the desired embedding size (must be a multiple of 8)
desired_embedding_size = 50264  # Change this to the desired size
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
### Resize the embedding layer to the desired size
model.resize_token_embeddings(len(tokenizer), desired_embedding_size)
model = model.to(device)

## save tokenizer and model to harddisk
tokenizer.save_pretrained(result_dir)
model.save_pretrained(result_dir)

In [ ]:
## load model and tokenizer from harddisk
### Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(result_dir)

### Load the GPT-2 model from the local folder
model = GPT2LMHeadModel.from_pretrained(result_dir)
model.to(device)


In [ ]:
# Define infer and train function
def infer(inp):
  inp = "<startofstring> " + inp + " <bot>: "
  inp = tokenizer(inp, return_tensors="pt")
  X = inp["input_ids"].to(device)  # Use .to(device) method to move the tensor to the specified device
  a = inp["attention_mask"].to(device)  # Use .to(device) method here as well

  output = model.generate(X, attention_mask=a, max_length=100, num_return_sequences=1)

  output = tokenizer.decode(output[0])

  return output

def train(chatData, model, optim):

  epochs = 12

  for _ in tqdm.tqdm(range(epochs)):  # Use range() to iterate through epochs
      for X, a in chatData:
          print(X)
          X = X.to(device)
          a = a.to(device)
          optim.zero_grad()
          loss = model(input_ids=X, attention_mask=a, labels=X).loss
          loss.backward()
          optim.step()

  # Save the model's state dictionary after training is complete
  torch.save(model.state_dict(), "model_state.pt")
  print(infer("How do you see the integration of holographic technology in education?"))




In [ ]:
from chat_data import ChatData

#Load ChatData, train model and optimizer
chatData = ChatData(data_file_path, tokenizer)
chatData = DataLoader(chatData, batch_size=64)

model.train()

optim = Adam(model.parameters())

In [ ]:
# train 10 times
epochs = 10  # You can adjust the number of epochs as needed
for epoch in range(epochs):
    print("Round: ", epoch)
    train(chatData, model, optim)

In [ ]:
inp = ""
while True:
  inp = input("Enter your input (press Enter when done): " + " " * 20)
  print(infer(inp))